Importación de librerias a usar

In [1]:
!pip install PyPDF2
import PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 9.5 MB/s eta 0:00:00


In [5]:
import os
import re
import pandas as pd
from PyPDF2 import PdfReader
import nltk
nltk.download('stopwords')
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
import time
from sklearn.metrics.pairwise import cosine_similarity
import csv
nltk.download('punkt')
import unicodedata
from nltk.tokenize import word_tokenize
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Extracción del texto de un pdf

In [6]:
import os
import re
import pandas as pd
from PyPDF2 import PdfReader

# Función para extraer texto de un PDF
def extraer_texto(pdf_path):
    texto = ""
    with open(pdf_path, "rb") as file:
        reader = PdfReader(file)
        for page in reader.pages:
            texto += page.extract_text()
    return texto

# Función para encontrar títulos y el texto entre ellos
def encontrar_titulos_y_texto(texto):
    # Patrón para detectar los títulos
    patron_titulo = r'(\d{2}/\d{2}/\d{4} \d{2}:\d{2} [A-ZÁÉÍÓÚÑ\s]+ \([A-ZÁÉÍÓÚÑ\s]+\))'
    bloques = re.split(patron_titulo, texto)
    # Eliminar el primer elemento vacío
    bloques = bloques[1:]
    # Agrupar títulos y texto entre ellos
    bloques = [bloques[i:i+2] for i in range(0, len(bloques), 2)]
    return bloques

# Función para eliminar fechas, números y caracteres no alfanuméricos
def eliminar_fechas_numeros_no_alfanumericos(texto):
    patron_fechas = r'\d{2}/\d{2}/\d{4}'  # Coincide con fechas en formato dd/mm/yyyy
    texto_sin_fechas = re.sub(patron_fechas, '', texto)

    patron_numeros = r'\d+'  # Coincide con números
    texto_sin_numeros = re.sub(patron_numeros, '', texto_sin_fechas)

    texto_filtrado = re.sub(r'[^a-zA-ZáéíóúÁÉÍÓÚñÑ\s]', '', texto_sin_numeros)
    return texto_filtrado

# Función para eliminar los pies de página
def eliminar_pie_pagina(texto):
    # Patrón para eliminar el pie de página
    patron_pie_pagina = r'Página \d+ de \d+'
    texto_sin_pie = re.sub(patron_pie_pagina, '', texto)
    return texto_sin_pie

# Función para filtrar la información del veredicto
def filter_verdict_information(texto):
    # Patrones para excluir información sobre el veredicto
    patrones_veredicto = [
        r'culpable',  # coincide con "culpable"
        r'inocente',  # coincide con "inocente"
        r'verdicto',  # coincide con "veredicto"
        r'sentencia',  # coincide con "sentencia"
        r'condena',  # coincide con "condena"
        r'absuelto',  # coincide con "absuelto"
        r'libertad',  # coincide con "libertad"
        r'culpabilidad',  # coincide con "culpabilidad"
        r'inocencia'  # coincide con "inocencia"
        # Agregar más patrones según sea necesario
    ]

    # Patrón para eliminar correos electrónicos
    patron_correo = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'

    # Combinar los patrones en una sola expresión regular
    patron_combinado = re.compile('|'.join(patrones_veredicto + [patron_correo]), re.IGNORECASE)

    # Eliminar todas las coincidencias
    texto_filtrado = patron_combinado.sub('', texto)

    return texto_filtrado

# Directorio con archivos PDF
pdf_directory = "/content/Archivos"

# Listar todos los archivos PDF en el directorio
pdf_files = [f for f in os.listdir(pdf_directory) if f.endswith('.pdf')]

data = []

# Procesar cada archivo PDF
for pdf_file in pdf_files:
    pdf_path = os.path.join(pdf_directory, pdf_file)
    # Extraer texto del PDF
    texto_pdf = extraer_texto(pdf_path)

    # Encontrar títulos y el texto entre ellos
    titulos_y_texto = encontrar_titulos_y_texto(texto_pdf)
    for titulo, texto in titulos_y_texto:
        texto_sin_pie = eliminar_pie_pagina(texto)
        texto_filtrado = filter_verdict_information(texto_sin_pie)
        texto_final = eliminar_fechas_numeros_no_alfanumericos(texto_filtrado)
        data.append((pdf_file, titulo, texto_final.strip()))

# Crear un DataFrame y guardarlo en un archivo CSV
df = pd.DataFrame(data, columns=['NombreArchivo', 'Título', 'Texto'])
df.to_csv('documento_procesado.csv', index=False)
print(df)


    NombreArchivo                                             Título  \
0           1.pdf       03/07/2019 14:35 RAZON DE EJECUTORIA (RAZON)   
1           1.pdf  18/04/2019 08:07 PROVIDENCIA GENERAL (RAZON DE...   
2           1.pdf     17/04/2019 10:43 PROVIDENCIA GENERAL (DECRETO)   
3           1.pdf  11/04/2019 17:05 SENTENCIA CONDENATORIA (RAZON...   
4           1.pdf  11/04/2019 16:05 SENTENCIA CONDENATORIA (RESOL...   
..            ...                                                ...   
642         8.pdf                   12/10/2022 12:04 OFICIO (OFICIO)   
643         8.pdf  07/10/2022 15:13 CONVOCATORIA AUDIENCIA DE JUI...   
644         8.pdf  07/10/2022 11:36 CONVOCATORIA AUDIENCIA DE JUI...   
645         8.pdf                     29/09/2022 13:11 RAZON (RAZON)   
646         8.pdf                     29/09/2022 13:09 RAZON (RAZON)   

                                                 Texto  
0    RAZON RAZÓN Esmeraldas  de Julio del  a las H ...  
1    En Esmeraldas mi

Convirtiendo los documentos a un DataFrame por secciones

In [8]:
import pandas as pd

# Suponiendo que 'df' es tu DataFrame original

# Convertir la columna 'NombreArchivo' a string
df['NombreArchivo'] = df['NombreArchivo'].astype(str)

# Filtrar filas que contienen 'RESOLUCION' o 'AUTO RESOLUTIVO' en la columna 'Título'
condicion = df['Título'].str.contains('RESOLUCION') | df['Título'].str.contains('AUTO RESOLUTIVO')
filas_seleccionadas = df[condicion]

# Función para asignar sentencia basada en el nombre del archivo
def asignar_sentencia(nombre_archivo):
    numero = int(nombre_archivo.split('.')[0])
    return 1 if 0 <= numero <= 9 else 0

# Unir textos de filas con títulos repetidos
filas_unidas = df.groupby('NombreArchivo')['Texto'].apply(' '.join).reset_index()

# Asignar valores de sentencia
filas_unidas['sentencia'] = filas_unidas['NombreArchivo'].apply(asignar_sentencia)

# Extraer la parte numérica del nombre del archivo para la ordenación
filas_unidas['NombreArchivo_num'] = filas_unidas['NombreArchivo'].str.extract(r'(\d+)', expand=False).astype(int)

# Ordenar el DataFrame 'filas_unidas' por 'NombreArchivo_num'
filas_unidas = filas_unidas.sort_values(by='NombreArchivo_num').reset_index(drop=True)

# Eliminar la columna auxiliar 'NombreArchivo_num'
filas_unidas.drop(columns='NombreArchivo_num', inplace=True)

# Guardar las filas unidas en una variable
filas_unidas_var = filas_unidas['Texto']

# Convertir la columna 'NombreArchivo' a un tipo que pandas puede ordenar correctamente en el DataFrame original
df['NombreArchivo_num'] = df['NombreArchivo'].str.extract(r'(\d+)', expand=False).astype(int)

# Ordenar el DataFrame por 'NombreArchivo_num'
df_sorted = df.sort_values(by='NombreArchivo_num').reset_index(drop=True)

# Convertir 'NombreArchivo_num' de nuevo a formato original con '.pdf'
df_sorted['NombreArchivo'] = df_sorted['NombreArchivo_num'].astype(str) + '.pdf'

# Eliminar la columna auxiliar 'NombreArchivo_num'
df_sorted.drop(columns='NombreArchivo_num', inplace=True)

# Imprimir el DataFrame ordenado y con la columna 'sentencia'
print(filas_unidas)


   NombreArchivo                                              Texto  sentencia
0          1.pdf  RAZON RAZÓN Esmeraldas  de Julio del  a las H ...          1
1          2.pdf  En Quito martes doce de marzo del dos mil vein...          1
2          3.pdf  En Quito jueves dieciséis de febrero del dos m...          1
3          4.pdf  En Riobamba miércoles diecisiete de mayo del d...          1
4          5.pdf  En Quito viernes veinte de octubre del dos mil...          1
5          6.pdf  En Quito martes treinta y uno de enero del dos...          1
6          7.pdf  Quito lunes  de mayo del  a las h VISTOS En mi...          1
7          8.pdf  En Quito jueves siete de septiembre del dos mi...          1
8          9.pdf  En Quito lunes veinte y siete de mayo del dos ...          1
9         10.pdf  En consecuencia en aplicación del derecho a la...          0
10        11.pdf  Dando cumplimiento a lo dispuesto en  de  de a...          0
11        12.pdf  RAZON Siento como tal que dando cu

Limpiar los textos

In [10]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [11]:
# Lista de nombres comunes y títulos en español
titulos = ["sr", "sra", "dr", "dra", "lic", "ing", "prof", "secretario", "director", "ab", "senor", "senora"]
numeros_como_palabras = ["hora","minuto","uno", "dos", "tres", "cuatro", "cinco", "seis", "siete", "ocho", "nueve", "diez", "once", "doce", "trece", "catorce", "quince", "dieciséis", "diecisiete", "dieciocho", "diecinueve", "veinte", "veintiuno", "veintidós", "veintitrés", "veinticuatro", "veinticinco", "veintiséis", "veintisiete", "veintiocho", "veintinueve", "treinta", "cuarenta", "cincuenta", "sesenta", "setenta", "ochenta", "noventa", "cien"]

# Lista de meses
meses = ["enero", "febrero", "marzo", "abril", "mayo", "junio", "julio", "agosto", "septiembre", "octubre", "noviembre", "diciembre"]

# Lista de días de la semana
dias_semana = ["lunes", "martes", "miércoles", "jueves", "viernes", "sábado", "domingo"]

# Función para eliminar nombres utilizando una lista de nombres comunes y títulos
def eliminar_nombres(texto):
    # Normalizar el texto para eliminar acentos y tildes
    texto = unicodedata.normalize('NFKD', texto)
    texto = "".join([c for c in texto if not unicodedata.combining(c)])

    # Convertir el texto a minúsculas
    texto = texto.lower()

    for hora in numeros_como_palabras:
        texto = re.sub(r'\b{}\b \w+'.format(hora), '', texto)

    # Eliminar títulos y los nombres que les siguen
    for mes in meses:
        texto = re.sub(r'\b{}\b \w+'.format(mes), '', texto)

    for dia in dias_semana:
        texto = re.sub(r'\b{}\b \w+'.format(dia), '', texto)

    for titulo in titulos:
        texto = re.sub(r'\b{}\b \w+'.format(titulo), '', texto)

    return texto

def normalizar_texto(texto):
    # Normalizar caracteres eliminando acentos y tildes
    texto = unicodedata.normalize('NFKD', texto)
    texto = "".join([c for c in texto if not unicodedata.combining(c)])
    return texto

def clean_text(*args):
    regex_date = r'\b\d{1,2}/\d{1,2}/\d{2,4}\b'  # Expresión regular para eliminar fechas en formato dd/mm/yyyy
    regex_number = r'\b\d+\b'  # Expresión regular para eliminar números
    regex_non_alnum = r'[^a-zA-ZáéíóúÁÉÍÓÚñÑ\s]'  # Expresión regular para eliminar caracteres no alfanuméricos
    stop_words = set(stopwords.words('spanish'))  # Obtener palabras vacías en español
    stemmer = PorterStemmer()  # Inicializar el objeto para realizar la derivación (stemming)
    cleaned_texts = []  # Lista para almacenar los textos limpios

    for text in args:
        cleaned_text = normalizar_texto(text)  # Normalizar texto para eliminar acentos y tildes
        cleaned_text = eliminar_nombres(cleaned_text)  # Eliminar nombres comunes y títulos
        cleaned_text = re.sub(regex_date, '', cleaned_text)  # Eliminar fechas
        cleaned_text = re.sub(regex_number, '', cleaned_text)  # Eliminar números
        cleaned_text = re.sub(regex_non_alnum, ' ', cleaned_text)  # Eliminar caracteres no alfanuméricos
        cleaned_text = cleaned_text.lower()  # Convertir texto a minúsculas
        tokens = word_tokenize(cleaned_text)  # Tokenizar texto
        tokens = [word for word in tokens if word not in stop_words]  # Eliminar palabras vacías
        tokens = [stemmer.stem(word) for word in tokens]  # Aplicar derivación a las palabras
        tokens = [word for word in tokens if len(word) > 1]  # Eliminar palabras de una sola letra
        cleaned_texts.append(" ".join(tokens))  # Unir las palabras limpias en un solo texto

    return cleaned_texts
d =clean_text(*filas_unidas['Texto'])
print(d)

['razon razon esmeralda siento tal auto anteced present proceso encuentra ejecutoriado ministerio ley remit proceso departamento archivo central certifico bonaga yagual esmeralda miercol partir mediant boleta judicial notifiqu decreto anteced sanchez perea eduard alberto casilla quinonez angulo francisco xavier casilla correo electronico casilla correo electronico casillero electronico drab jhonni fernando ronquillo cuero certifico esmeralda miercol princip agregues proceso oficio ujp fecha suscrito mendoza aria coordinadora unidad judici penal esmeralda indica angulo francisco xavier realizo trabajo comunitario unidad dando cumplimiento ordenado autoridad actu bonaga yagual despacho cumplas notifiqu esmeralda juev partir mediant boleta judicial notifiqu anteced sanchez perea eduard alberto casilla quinonez angulo francisco xavier casilla correo electronico casilla correo electronico casillero electronico drab jhonni fernando ronquillo cuero certifico esmeralda juev visto suscrito coro

Prediccion del modelo

In [12]:
# Supongamos que tienes dos listas de textos preprocesados: documentos_culpables y documentos_inocentes
documentos_culpables = filas_unidas.loc[0:9, 'Texto'].tolist()  # Llena esto con tus textos preprocesados de documentos culpables
documentos_inocentes = filas_unidas.loc[10:, 'Texto'].tolist()  # Llena esto con tus textos preprocesados de documentos inocentes

cdc = clean_text(*documentos_culpables)
cdi = clean_text(*documentos_inocentes)

#Unir ambos conjuntos para ajustar el vectorizador a todos los documentos
todos_los_documentos =  cdc  + cdi

# # Crear un TF-IDF vectorizer y ajustar a todos los documentos
vectorizer = TfidfVectorizer()  # Puedes ajustar el número de características según sea necesario
vectorizer.fit(todos_los_documentos)
# Transformar cada conjunto de documentos

# # Transformar cada conjunto de documentos
tfidf_culpables = vectorizer.transform(documentos_culpables)
tfidf_inocentes = vectorizer.transform(documentos_inocentes)

# Obtener los nombres de las características
palabras = vectorizer.get_feature_names_out()

# Convertir las matrices TF-IDF a DataFrames para mejor manejo
df_tfidf_culpables = pd.DataFrame(tfidf_culpables.toarray(), columns=palabras)
df_tfidf_inocentes = pd.DataFrame(tfidf_inocentes.toarray(), columns=palabras)

# Promediar los TF-IDF para cada palabra en ambos grupos
promedio_tfidf_culpables = df_tfidf_culpables.mean(axis=0)
promedio_tfidf_inocentes = df_tfidf_inocentes.mean(axis=0)


promedio_tfidf_culpables.to_csv('documento_procesado.csv', index=False)


# Función para calcular la similitud coseno entre un nuevo documento y los promedios TF-IDF de culpables e inocentes
def calcular_similitud(nuevo_documento, vectorizer, promedio_tfidf_culpables, promedio_tfidf_inocentes):
    # Preprocesar el nuevo documento
    nuevo_documento_procesado = clean_text(nuevo_documento)[0]  # Ajustar para procesar un solo documento

    # Transformar el documento usando el vectorizador TF-IDF
    nuevo_documento_tfidf = vectorizer.transform([nuevo_documento_procesado])

    # Calcular la similitud coseno con los promedios de culpables e inocentes
    similitud_culpables = cosine_similarity(nuevo_documento_tfidf, [promedio_tfidf_culpables])
    similitud_inocentes = cosine_similarity(nuevo_documento_tfidf, [promedio_tfidf_inocentes])

    return similitud_culpables[0][0], similitud_inocentes[0][0]

# Crear una matriz de distancias
matriz_distancias = np.zeros((20, 2))

# Iterar sobre los documentos en el vector d del 0 al 19
for i in range(20):
    nuevo_documento = d[i]
    similitud_culpables, similitud_inocentes = calcular_similitud(nuevo_documento, vectorizer, promedio_tfidf_culpables, promedio_tfidf_inocentes)
    matriz_distancias[i, 0] = similitud_culpables
    matriz_distancias[i, 1] = similitud_inocentes

# Convertir la matriz de distancias a un DataFrame para mejor visualización
df_matriz_distancias = pd.DataFrame(matriz_distancias, columns=['Similitud Culpables', 'Similitud Inocentes'])

# # Imprimir la matriz de distancias
print(df_matriz_distancias)

# Determinar la clasificación basada en la similitud más alta
resultados = []
for i in range(20):
    if matriz_distancias[i, 0] > matriz_distancias[i, 1]:
        resultado = "Culpable"
    else:
        resultado = "Inocente"
    resultados.append((i, resultado))

# Imprimir los resultados
for idx, resultado in resultados:
    print(f"El documento {idx} es: {resultado}")




    Similitud Culpables  Similitud Inocentes
0              0.318928             0.269268
1              0.382667             0.261845
2              0.333672             0.205644
3              0.344335             0.221256
4              0.369149             0.301635
5              0.396338             0.304771
6              0.427468             0.295301
7              0.367232             0.283930
8              0.386885             0.268606
9              0.424166             0.316282
10             0.303972             0.428570
11             0.231661             0.355953
12             0.279791             0.377860
13             0.379467             0.467736
14             0.262851             0.399726
15             0.281389             0.416873
16             0.278940             0.407838
17             0.312208             0.443401
18             0.288909             0.423323
19             0.277164             0.400281
El documento 0 es: Culpable
El documento 1 es: Culpable

Prediccion nuevo documento

In [ ]:
import os
import re
import pandas as pd
from PyPDF2 import PdfReader
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('stopwords')

# Función para extraer texto de un PDF
def extraer_texto(pdf_path):
    texto = ""
    with open(pdf_path, "rb") as file:
        reader = PdfReader(file)
        for page in reader.pages:
            texto += page.extract_text()
    return texto


# Crear un vectorizador de conteo para convertir tokens en una matriz
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(cdi + cdc)

# Función para calcular la similitud coseno entre un nuevo documento y los promedios TF-IDF de culpables e inocentes
def calcular_similitud(nuevo_documento, vectorizer, promedio_tfidf_culpables, promedio_tfidf_inocentes):
    # Preprocesar el nuevo documento
    nuevo_documento_procesado = clean_text(nuevo_documento)[0]  # Ajustar para procesar un solo documento

    # Transformar el documento usando el vectorizador TF-IDF
    nuevo_documento_tfidf = vectorizer.transform([nuevo_documento_procesado])

    # Calcular la similitud coseno con los promedios de culpables e inocentes
    similitud_culpables = cosine_similarity(nuevo_documento_tfidf, [promedio_tfidf_culpables])
    similitud_inocentes = cosine_similarity(nuevo_documento_tfidf, [promedio_tfidf_inocentes])
    es_culpable = similitud_culpables.mean() > similitud_inocentes.mean()
    return es_culpable

# Función para clasificar un nuevo documento
def clasificar_documento(nuevo_documento, vectorizer, promedio_tfidf_culpables, promedio_tfidf_inocentes):
    similitud_culpables = calcular_similitud(nuevo_documento, vectorizer, promedio_tfidf_culpables, promedio_tfidf_inocentes)
    return "El documento es culpable." if similitud_culpables  else "El documento es inocente."

# Ejemplo de uso: ingresa la ruta de tu nuevo documento PDF aquí
ruta_nuevo_documento = "/content/expel_13266202100174_23027690_02062024.pdf"
texto_nuevo_documento = extraer_texto(ruta_nuevo_documento)
resultado = clasificar_documento(texto_nuevo_documento, vectorizer, promedio_tfidf_culpables, promedio_tfidf_inocentes)
print(f"El documento es: {resultado}")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


El documento es: El documento es inocente.


In [13]:
# Ejemplo de tokens de documentos inocentes y culpables
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
# Crear un vectorizador de conteo para convertir tokens en una matriz
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(cdi + cdc)

# Convertir el texto del nuevo documento en tokens y transformarlo a una matriz
ruta_nuevo_documento = "/content/Archivos/8.pdf"
nuevo_documento = extraer_texto(ruta_nuevo_documento)
nuevo_documento_procesado = clean_text(nuevo_documento)[0]
print(nuevo_documento_procesado)
tokens_nuevo_documento = vectorizer.transform([nuevo_documento_procesado])

# Calcular la similitud coseno entre el nuevo documento y los documentos inocentes y culpables
similitud_inocente1 = cosine_similarity(tokens_nuevo_documento, X[:len(cdi)])
similitud_culpable1 = cosine_similarity(tokens_nuevo_documento, X[len(cdc):])
# Ver si la similitud con los documentos culpables es mayor que con los inocentes
es_culpable = similitud_culpable1.mean() > similitud_inocente1.mean()
print(similitud_culpable1.mean() , similitud_inocente1.mean())
print("El documento es culpable." if es_culpable else "El documento es inocente.")


republica ecuador www funcionjudici gob ec tribun garantia penal sede parroquia inaquito distrito metropolitano quito provincia pichincha proceso ingreso tipo materia penal coip tipo accion procedimiento accion penal publica tipo asunto delito lesion inc tercero cuarto actor ofendido emilio marcelo simba remach fiscalia gener demandado procesado remach teran edwin omar providencia gener razon notificacion quito juev tre partir mediant boleta judicial notifiqu decreto anteced defensoria publica casillero electronico correo electronico defensajudici defensoria gob ec dr ab defensoria publica penal patrocinio quito defensoria publica casillero casillero electronico correo electronico defensadeoficiopichincha defensoria gob ec dr ab defensoria publica penal oficio quito emilio marcelo simba remach casillero casillero electronico correo electronico victimaspichincha defensoria gob ec dr ab defensoria publica pichincha victima fiscalia gener casillero casillero electronico correo electronico